## Xarray engine: writing back to GRIB

<div class="alert alert-warning">
<b>Warning:</b> converting back Xarray to GRIB is an experimental feature and is not yet fully supported. </div>

First, we get some example GRIB data and convert it into Xarray.

In [1]:
import earthkit.data as ekd

ds_fl = ekd.from_source("sample", "pl.grib")
ds_xr = ds_fl.to_xarray()
ds_xr

pl.grib:   0%|          | 0.00/48.8k [00:00<?, ?B/s]

<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, level: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * level                    (level) int64 16B 500 700
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
Attributes:
    class:        od
    stream:       oper
    levtype:      pl
    type:         fc
    expver:       0001
    date:         20240603
    time:         0
    domain:       g
    number:       0
    Conventions:  CF-1.8
    institution:  ECMWF

In [2]:
# ensure earthkit attributes are set
import xarray as xr 
xr.set_options(keep_attrs=True)

# modify values
ds_xr += 1

#### Using to_target()

First, we write a datarray into a GRIB file.

In [3]:
# option1: writing to GRIB file using the accessor
ds_xr["t"].earthkit.to_target("file", "_from_xr_1.grib")

# option2: writing to GRIB file using the top level function
ekd.to_target("file", "_from_xr_1a.grib", data=ds_xr["t"])

# check the results
ds_tmp1 = ekd.from_source("file", "_from_xr_1.grib")
ds_tmp1[0]

GribField(t,500,20240603,0,0,0)

Next, we write the whole dataset into a GRIB file.

In [4]:
# option1: writing to GRIB file using the accessor
ds_xr.earthkit.to_target("file", "_from_xr_2.grib")

# option2: writing to GRIB file using the top level function
ekd.to_target("file", "_from_xr_2a.grib", data=ds_xr)

# check the results
ds_tmp2 = ekd.from_source("file", "_from_xr_2.grib")
ds_tmp2.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


We check if the computation results were correctly written to the generated GRIB data.

In [5]:
# original GRIB data
print(ds_fl.sel(param="t", step=0, level=500)[0].values[0])
# GRIB data converted from the modified xarray object
print(ds_tmp1.sel(param="t", step=0, level=500)[0].values[0])

250.22500610351562
251.22500610351562


#### Using to_fieldlist()

First, we convert a dataarray to a GRIB fieldlist.

In [6]:
ds_fl1 = ds_xr["t"].earthkit.to_fieldlist()
ds_fl1.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,t,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,t,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,t,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,t,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


Next, we convert back the whole dataset into a GRIB fieldlist.

In [7]:
ds_fl2 = ds_xr.earthkit.to_fieldlist()
ds_fl2.head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,r,isobaricInhPa,500,20240603,0,6,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll
4,ecmf,r,isobaricInhPa,500,20240603,1200,0,fc,0,regular_ll


In [8]:
out_name = "_from_xr_3.grib"
ds_fl1.to_target("file", out_name)
# read back and check the saved GRIB
ds_tmp = ekd.from_source("file", out_name)
ds_tmp[0]

GribField(t,500,20240603,0,0,0)